In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano')

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

chain.invoke('더블딥')

'더블딥은 한 차례 경기 침체가 끝난 뒤 짧은 회복을 보이다가 다시 침체에 빠지는 경제 현상입니다. 즉, GDP가 두 차례 연속으로 감소하는 구간이 사이에 짧은 확장기가 있는 형태를 말합니다.\n\n주요 특징\n- 두 차례의 하강이 서로 간격을 두고 반복됩니다.\n- 회복기가 비교적 짧거나 연약하게 나타날 수 있습니다.\n\n주요 원인과 시사점\n- 원인: 수요 부진, 고용 악화, 정책 불확실성, 외부 충격 등이 복합적으로 작용합니다.\n- 시사점: 신속한 재정·금융 정책 대응으로 회복의 지속성을 확보하는 것이 중요합니다. \n\n다른 뜻이 있다면 알려 주세요.'

In [8]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-4o-mini', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [9]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream('더블딥'):
    print(chunk, end='', flush=True)

"더블 딥(Double Dip)"은 경제학에서 사용되는 용어로, 경제가 일시적으로 회복한 후 다시 침체에 빠지는 상황을 가리킵니다. 일반적으로 경기 침체가 발생한 후 기업과 소비자 신뢰가 회복되어 경제 활동이 증가하지만, 이 회복세가 오래가지 않고 다시 부정적인 경과로 이어지는 경우를 의미합니다. 이러한 현상은 경제의 불확실성, 외부 충격, 정치적 불안정 등 다양한 요인으로 인해 발생할 수 있습니다. 더블 딥은 경제 정책에 있어 큰 도전 과제가 될 수 있습니다.

In [12]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [13]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[0])
print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[1])

더블딥은 경제 용어로, 경기 침체가 끝나가나 싶었는데 다시 하강해 이중으로 침체에 빠지는 현상을 말합니다. 즉, 짧은 회복 국면 후 다시 경기 하강이 나타나는 상황을 가리키는 표현입니다. 추가로 특정 분야(예: 금융, 정책)에 대한 설명이 필요하면 알려주세요.
인플레이션은 일정 기간 물가가 지속적으로 올라가는 현상으로, 화폐의 구매력이 떨어지는 것을 말합니다.

주요 포인트
- 원인: 수요 증가에 따른 수요견인 인플레이션, 생산비 증가 등 비용 인상으로 생기는 비용인플레이션, 그리고 기대 인플레이션.
- 측정: 물가상승률을 주로 소비자물가지수(CPI)로 판단합니다.
- 관리: 중앙은행의 금리 조정 등 통화정책으로 물가 상승을 억제하거나 관리합니다.
- 영향: 일반 가계의 실질구매력 감소, 저축 가치 하락, 채무자에겐 부담 경감 등으로 다양한 경제적 효과가 있습니다.
- 간단한 예: 물가가 연간 3% 상승하면 같은 돈으로 살 수 있는 물건의 양이 점차 줄어듭니다.
